In [3]:
import torch
from torch import nn
from typing import Tuple


def vgg_block(n_convs: int, in_ch: int, out_ch: int) -> nn.Sequential:
    layers = list()
    for _ in range(n_convs):
        layers.append(nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_ch = out_ch
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)


def vgg(
    conv_arch: Tuple[Tuple[int, int], ...],
    in_channels: int = 1,
    in_dims: Tuple[int, int] = (244, 244),
    n_classes: int = 10,
) -> nn.Sequential:
    conv_blks = []
    # The convolutional part
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
        in_dims = (in_dims[0] // 2, in_dims[1] // 2)

    return nn.Sequential(
        *conv_blks,
        nn.Flatten(),
        # The fully-connected part
        nn.Linear(out_channels * in_dims[0] * in_dims[1], 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, n_classes)
    )

In [4]:
from src.data import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

data_root = "../data/raw/imagenette2-320/"

dims = (64, 64)
trans = transforms.Compose(
    [
        transforms.Resize(dims),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

training_set = datasets.Imagenette(
    data_root + "noisy_imagenette.csv", data_root, train=True, transform=trans
)
validation_set = datasets.Imagenette(
    data_root + "noisy_imagenette.csv", data_root, train=False, transform=trans
)

train_loader = DataLoader(training_set, batch_size=4, shuffle=True, num_workers=2)
valid_loader = DataLoader(validation_set, batch_size=4, shuffle=True, num_workers=2)

conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
net = vgg(conv_arch, in_channels=3, in_dims=dims, n_classes=10)

In [14]:
import torch
import torch.nn as nn
from torch.autograd import Variable

output = Variable(torch.FloatTensor([0,0,1,1])).view(1, -1)
target = Variable(torch.LongTensor([3]))

criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss)

tensor(1.0064)


In [11]:
def train_net(net, trainloader, num_epochs, lr, device):
    net.to(device)
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    
    import torch.optim as optim

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')

In [12]:
lr, num_epochs = 0.01, 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_net(net, train_loader, num_epochs, lr, device)

[1,  2000] loss: 2.298
[2,  2000] loss: 2.265
[3,  2000] loss: 2.083
[4,  2000] loss: 1.910
[5,  2000] loss: 1.762
[6,  2000] loss: 1.628
[7,  2000] loss: 1.535
[8,  2000] loss: 1.424
[9,  2000] loss: 1.342
[10,  2000] loss: 1.243
Finished Training


In [33]:
m = nn.Softmax(dim=1)

i = 0
for x, y in valid_loader:
    X = x.to(device)
    logits = net(X)
    pred_probab = nn.Softmax(dim=1)(logits)
    y_pred = pred_probab.argmax(1)
    print(y_pred)
    print(y)
    if i > 20:
        break
    i+=1


tensor([2, 2, 2, 2], device='cuda:0')
tensor([2, 4, 6, 2])
tensor([0, 3, 2, 5], device='cuda:0')
tensor([8, 0, 8, 5])
tensor([8, 2, 2, 6], device='cuda:0')
tensor([7, 4, 9, 6])
tensor([2, 4, 8, 5], device='cuda:0')
tensor([2, 4, 3, 4])
tensor([5, 2, 2, 4], device='cuda:0')
tensor([5, 2, 1, 4])
tensor([0, 5, 0, 2], device='cuda:0')
tensor([0, 5, 0, 9])
tensor([3, 7, 2, 1], device='cuda:0')
tensor([3, 4, 2, 1])
tensor([3, 6, 9, 1], device='cuda:0')
tensor([3, 0, 4, 1])
tensor([2, 2, 1, 6], device='cuda:0')
tensor([8, 3, 1, 6])
tensor([4, 1, 5, 2], device='cuda:0')
tensor([5, 4, 6, 2])
tensor([4, 0, 2, 0], device='cuda:0')
tensor([4, 0, 8, 8])
tensor([6, 3, 2, 3], device='cuda:0')
tensor([9, 9, 2, 0])
tensor([9, 2, 6, 5], device='cuda:0')
tensor([9, 2, 6, 5])
tensor([3, 5, 6, 4], device='cuda:0')
tensor([7, 8, 6, 4])
tensor([1, 2, 1, 2], device='cuda:0')
tensor([1, 8, 0, 2])
tensor([2, 4, 6, 1], device='cuda:0')
tensor([8, 4, 6, 1])
tensor([5, 2, 5, 5], device='cuda:0')
tensor([8, 7, 5, 7

In [34]:
torch.save(net.state_dict(), 'model_weights.pth')